In [2]:
import pandas as pd
import xlrd
import requests
import numpy as np
from bs4 import BeautifulSoup
import re
import fnmatch
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import json
import pickle
import csv
from datetime import datetime
from datetime import timedelta
from dateutil import parser
%matplotlib inline
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
import pytz

In [3]:
from patsy import dmatrices
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from texttable import Texttable

### Create dataframe of economic indicators by month Jan 1945 - Dec 2009

In [2]:
df_econ = pd.read_excel("Luther_excel.xlsx",set_index=True)

In [3]:
df_econ.columns = ['Month','USREC']
df_econ['Month'] = df_econ['Month'].map(lambda x: datetime.date(x))
df_econ.head()

,Month,USREC
0,1945-01-01,0
1,1945-02-01,0
2,1945-03-01,1
3,1945-04-01,1
4,1945-05-01,1


In [4]:
#### Create list of recessionary periods (by month)

rec_months = list(df_econ[df_econ['USREC']==1].Month)

### Scrape actors names, total gross and number of movies from Box Office Mojo

In [ ]:
actors_FULL,tot_gross_FULL,num_movies_FULL = [],[],[]
pages = [1,2,4,8]

for i in pages:

    url = "http://www.boxofficemojo.com/people/?view=Actor&pagenum="+str(i)+"&sort=sumgross&order=DESC&adjust_yr=2015&adjust_mo=&&p=.htm"
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page,"html.parser")

    #ACTORS
    for link in soup.find_all('a',attrs = {'href':re.compile('/chart/')}):
        [actors_FULL.append(str(a.text)) for a in link.find_all('b')]    

    #TOTAL GROSS
    for link in soup.find_all('td',attrs = {'align':"right"}):
        [tot_gross_FULL.append(g.text.replace("$","").replace(",","").replace("k","")) 
         for g in link.find_all('b')]
    
    #NUMBER OF MOVIES
    for n in soup.find_all('td')[12::7]:
        num_movies_FULL.append(int(n.text))

### Create dataframe with data from Box Office Mojo

In [ ]:
tot_gross_fl_FULL = [float(x) for x in tot_gross_FULL]
d = zip(actors_FULL,tot_gross_fl_FULL,num_movies_FULL)
df = pd.DataFrame(d)
df.columns = ['Actor','Total_Gross_2015_Dol','Num_Movies']
df = df.drop(df.index[726:])
df['Avg_Gross'] = (df['Total_Gross_2015_Dol']/df['Num_Movies']).astype(float)


### Scrape Oscar data from 1960 - 2014

In [292]:
categories = range(1061,1065)

for c in categories:
    url = "http://awardsdatabase.oscars.org/ampas_awards/BasicSearch"
    data = {'action': 'performSearch',
                'BSFromYear': 53,
                'BSToYear': 87,
                'BSCategory': c,
               'displayType': 1}
    response = requests.post(url, data=data)

    page = response.text
    soup = BeautifulSoup(page,"html.parser")

    for name in soup.find_all('a',attrs = {'href':re.compile('BSNominationID')}):
            osc_names.append(name.text)

### Scrape number of Oscar nominations for each actor

In [200]:
oscar_actors = list(df[df['Oscar']==1].Actor)

d = {actor: [] for actor in oscar_actors}
for actor in oscar_actors:    
#for c in categories:
    url = "http://awardsdatabase.oscars.org/ampas_awards/BasicSearch"
    data = {'action': 'performSearch',
                    'BSNominee': actor,    
                    'BSFromYear': 33,
                        'BSToYear': 87,
                        'BSCategory': 0,
                       'displayType': 6}
    response = requests.post(url, data=data)

    page = response.text
    soup = BeautifulSoup(page,"html.parser")

    for year in soup.find_all('a',attrs = {'href':re.compile('(?<=displayType=1&)BSFromYear')}):
            d[actor].append(year.text)

In [238]:
#### Convert years of Oscar nominations to dataframe and merge
df_oscar_yrs = pd.DataFrame(d.items())
df_oscar_yrs.columns = ['Actor','Years_Oscar']
df_oscar_yrs['Years_Oscar']=df_oscar_yrs['Years_Oscar'].map(lambda x: [year[:4] for year in x]) #extract only year
new_df = pd.merge(df,df_oscar_yrs,on='Actor',how='outer').fillna(0) #merge dataframes

In [277]:
#### Create new column for number of oscar nominations based on number of years
new_df['Years_Oscar'] = new_df['Years_Oscar'].map(lambda x: [str(unicode(i)) for i in x])
new_df['Num_Osc'] = new_df['Years_Oscar'].map(lambda x: len(x) if x!=0 else 0)

In [297]:
### Convert Oscar winners/nominees to strings
#### And create column based on Oscar win/nomination    
    
l_oscars = []

for name in osc_names:
    try:
        l_oscars.append(str(name))
    except:
        continue
            
df['Oscar']=df['Actor'].map(lambda x: 1 if x in l_oscars else 0)

In [ ]:
### Write birthday data for every actor in df to csv file birthdays.csv    

with open("birthdays.csv", 'a') as f: 
    writer=csv.writer(f)

    for actor in df.Actor[700:]:
        new_name = actor.replace(" ","_")
        url = "https://en.wikipedia.org/wiki/"+new_name

        response = requests.get(url)
        page = response.text
        soup = BeautifulSoup(page,"html.parser")

    #for line in soup.find_all('span',attrs = {'class':re.compile('BSNominationID')}):):
        try:
            for line in soup.find(class_="bday"):
                writer.writerow((actor,line))
        except:
            writer.writerow((actor,None))

In [ ]:
#### Strip white space off birthdays when reading csv data

def strip(text):
    try:
        return text.strip().replace('\n',", ")
    except AttributeError:
        return text

In [ ]:
#### Merge birthday data into dataframe

df_birthdays = pd.read_csv("birthdays.csv", converters={'Birthday':strip})
new_df = pd.merge(df,df_birthdays,on='Actor',how='outer')

### Scrape Wikipedia for occupation of each actor

In [ ]:
### Search for occupations on Wikipedia by Actor name

with open("occupations.csv", 'a') as f: 
    writer=csv.writer(f)

    for actor in df.Actor[700:]:
        new_name = actor.replace(" ","_")
        url = "https://en.wikipedia.org/wiki/"+new_name

        response = requests.get(url)
        page = response.text
        soup = BeautifulSoup(page,"html.parser")

        try:
            writer.writerow((actor,soup.find(class_="role").text))
        except:
            try:
                writer.writerow((actor,soup.find(scope_="Occupation").text))
            except:
                try: 
                    url = "https://en.wikipedia.org/wiki/"+new_name+"_(actor)"
                    response = requests.get(url)
                    page = response.text
                    soup = BeautifulSoup(page,"html.parser")
                    writer.writerow((actor,soup.find(class_="role").text))
                except:
                    try:
                        writer.writerow((actor,soup.find(scope_="Occupation").text))
                    except:
                        writer.writerow((actor,"None"))               

In [ ]:
#### Read Occupations data
df_occ = pd.read_csv("occupations.csv", converters={'Occupation':strip})

#### Create Gender columm
df_occ['Gender']=df_occ['Occupation'].map(lambda x: 0 if "actor" in x.lower() 
                                      else 1 if "actress" in x.lower() else "Unknown")

#### Dummy for people with Actors as principal occupation
df_occ['Occ_act_dummy'] = df_occ['Occupation'].map(lambda x: 1 if x.lower().startswith("actor") or x.lower().startswith("actress") else "None" if x=="None" else 0)

In [ ]:
#### Clean whitespaces off birthday    

df['Birthday'] = df['Birthday'].map(lambda x: str(x).replace(" ",""))

In [ ]:
#### Check for whitespace in birthdays

for n in df.Birthday:
    if type(n)==str:
        if len(n)>10:
            print n

In [ ]:
#### Create dummy to measure principal occupation as Actor (1) or otherwise (0)

new_df['Occ_act_dummy'] = new_df['Occupation'].map(lambda x: 1 if x.lower().startswith("actor") 
                                                   or x.lower().startswith("actress") else "None" if x=="None" else 0)

In [ ]:
#### Create Gender dummy based on occupation

new_df['Gender']=new_df['Occupation'].map(lambda x: 0 if "actor" in x.lower() 
                                          else 1 if "actress" in x.lower() else "Unknown")

In [16]:
#### Create age window, ages i to j

def age_range(bday,i=15,j=20):
    date_list = [datetime.date(datetime(bday.year,bday.month,1)) + 
                 relativedelta(months=x) for x in range(i*12, j*12)]
    return date_list

In [6]:
#### Read econ csv file
#### Count number of months in recessionary period in desired time range

df_econ = pd.read_excel("/Users/loraanngerson/Metis/github/webscrape-movie-data/data/Luther_excel.xlsx",set_index=True)
df_econ.columns = ['Month','USREC']
df_econ['Month'] = df_econ['Month'].map(lambda x: datetime.date(x))
rec_months = list(df_econ[df_econ['USREC']==1].Month)


def count_rec_pd(actors_dates):

    count = 0

    for d in actors_dates:
        if d in rec_months:
            count+=1 

    return count

In [8]:
#### Calculate age

def calc_age(bday):
    today = datetime.date(datetime.now(pytz.utc))
    return today.year - bday.year

In [208]:
#### Check if lists have elements in common

def lists_overlap(a, b):
    return bool(set(a) & set(b))

75

### Scrape Wikipedia for actor career length

In [189]:
active = []
for actor in actors_3:
    new_name = actor.replace(" ","_")
    url = "https://en.wikipedia.org/wiki/"+new_name

    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page,"html.parser")

    try:
        for line in soup.find(class_=re.compile('infobox')).find_all(string=re.compile('^Years')):
            active.append((actor,line.find_next().string))
    except:
        try: 
            url = "https://en.wikipedia.org/wiki/"+new_name+"_(actor)"
            response = requests.get(url)
            page = response.text
            soup = BeautifulSoup(page,"html.parser")
            for line in soup.find(class_=re.compile('infobox')).find_all(string=re.compile('^Years')):
                active.append((actor,line.find_next().string))
        except:
            active.append((actor,"None"))

In [ ]:
#### Create list of strings of "Years active" data   

l = []
for name,years in active:
    if years!=None:
        l.append((name,unicode(years).encode("ascii","ignore"))) # convert unicode to string
            
#### Separate "Years active" data into start and finish year 

list_years_active = []
for k,v in l:
    start,finish = v[:4],v[4:]
    list_years_active.append([k,start,finish])

In [ ]:
#### Check missing values

actors_3 = list(df[df['Career_start'].isnull()].Actor)
len(actors_3)

In [ ]:
#### Merged career start/finish data from separate scrape

for l in list_years_active:
    df.set_value(df[df['Actor']==l[0]].index, 'Career_start', l[1])

In [297]:
#### Convert date of career start to integer

df['Career_start'] = df['Career_start'].map(lambda x: int(x) if str(x).isdigit() else None)

In [254]:
#### Codeboook

codebook = {
    "Actor": "unique ID",
    "Total_Gross_2015_Dol": "Amount grossed by films featuring this actor",
    "Num_movies": "Number of movies used to calculate total gross",
    "Avg_Gross": "Total gross divided by number of movies",
    "Oscar": "Dummy variable 1 for Oscar win/nomination",
    "Birthday": "Actor's birthday (datetime object)",
    "Occupation": "Individual's occupation (source: Wikipedia)",
    "Gender": "Actor's gender based on Actor/Actress designation",    
    "Occ_act_dummy": "Dummy variable 1 for actor as primary occupation (source: Wikipedia)",
    "Num_months_rec_pd_15_20": "Count of months in US economic recession between ages 15-20",
    "Num_months_rec_pd_0_15": "Count of months in US economic recession between ages 0-15",
    "Age": "Age of actor",
    "Career_start": "Year acting career began (source: Wikipedia)",
    "Career_fin": "Year acting career ended (source: Wikipedia)"
}

In [10]:
#### New columns for number of economic recessions during certain age ranges
df['Num_months_rec_pd_0_5']=df['Birthday'].map(lambda x: count_rec_pd(age_range(x,0,5)))
df['Num_months_rec_pd_5_10']=df['Birthday'].map(lambda x: count_rec_pd(age_range(x,5,10)))
df['Num_months_rec_pd_10_15']=df['Birthday'].map(lambda x: count_rec_pd(age_range(x,10,15)))
df['Num_months_rec_pd_15_20']=df['Birthday'].map(lambda x: count_rec_pd(age_range(x,15,20)))
df['Num_months_rec_pd_20_25']=df['Birthday'].map(lambda x: count_rec_pd(age_range(x,20,25)))

In [24]:
df['Years_acting']=df['Age']-df['Age_start'] 
#New column for length of career

df_only_actors['Age_start'] = df_only_actors['Career_start'] - (2015 - df_only_actors['Age']) 
#New column for starting age

In [ ]:
#### Remove rows with NaN values in one column

df_2 = df[np.isfinite(df['Age_start'])]

In [ ]:
#### Seaborn histogram to plot distribution

sns.distplot(df.Age,bins=20)

In [ ]:
#### Four sublots of bar graphs 

fig, axes = plt.subplots(nrows=2, ncols=2,figsize=(15,10),sharex=True,sharey=True)
axes[0,0].bar(df.Num_months_rec_pd_0_5, df.Avg_Gross,width,color='y',label="Age 0-5")
axes[0,1].bar(df.Num_months_rec_pd_5_10, df.Avg_Gross, width, color='r', label="Age 5-10")
axes[1,0].bar(df.Num_months_rec_pd_10_15, df.Avg_Gross, width, color = 'g',label="Age 10-15")
axes[1,1].bar(df.Num_months_rec_pd_15_20, df.Avg_Gross, width, label="Age 15-20")
plt.legend()